In [3]:
import pandas as pd
import numpy as np
import folium
import os
import zipfile
import shutil
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import Point
from shapely.ops import unary_union
from folium.vector_layers import Polygon

In [5]:
# 엑셀 파일 불러오기
hospital_df = pd.read_excel("C:/Users/hk522/Desktop/캡스톤 팀프로젝트/병원 풀데이터 지번주소.xlsx")
pharmacy_df = pd.read_excel("C:/Users/hk522/Desktop/캡스톤 팀프로젝트/약국 풀데이터 지번주소.xlsx")

In [6]:
# 병원/약국 구분 컬럼 추가
hospital_df["구분"] = "병원"
pharmacy_df["구분"] = "약국"

# 병원 + 약국 데이터 합치기
full_df = pd.concat([hospital_df, pharmacy_df], ignore_index=True)

In [8]:
# 시간을 "HHMM" 형태에서 "H:MM" 형식으로 변환하는 함수
def format_time(val):
    try:
        val = int(val)
        hours = val // 100
        minutes = val % 100
        return f"{hours}:{minutes:02}"
    except:
        return None

In [9]:
# 마커에 들어갈 팝업 내용 생성 함수
def generate_popup(row):
    popup_html = f"<b>{row['dutyName']}</b>"

    if pd.notna(row.get("dutyTel1")) and row["dutyTel1"] != '':
        popup_html += f"<br>☎ {row['dutyTel1']}<br><br>"
    else:
        popup_html += "<br><br>"

    # <요일별 진료시간> 출력
    popup_html += "&lt;요일별 진료시간&gt;<br>"

    days = ['월', '화', '수', '목', '금', '토', '일', '공휴일']

    for i, day in enumerate(days, start=1):
        start_col = f'dutyTime{i}s'
        end_col = f'dutyTime{i}c'

        start = row.get(start_col)
        end = row.get(end_col)

        start_fmt = format_time(start)
        end_fmt = format_time(end)

        # "요일"이라는 접미사는 공휴일에는 붙이지 않음
        label = f"{day}요일" if day != "공휴일" else "공휴일"

        if start_fmt and end_fmt:
            popup_html += f"{label} : {start_fmt} ~ {end_fmt}<br>"
        else:
            popup_html += f"{label} : 휴진<br>"

    return popup_html

In [10]:
# 마커 아이콘 색상과 모양 설정
def get_icon(category):
    if category == "병원":
        return folium.Icon(color="red", icon="medkit", prefix='fa')   # 십자가 느낌의 의료 아이콘
    else:
        return folium.Icon(color="green", icon="pills", prefix='fa')  # 알약 모양 아이콘

In [ ]:
# 지도 생성 함수
def create_maps_by_region(df, grouped, output_dir='캡스톤 법정동 지도'):
    os.makedirs(output_dir, exist_ok=True)

    for region, group in grouped:
        valid_group = group.dropna(subset=["wgs84Lat", "wgs84Lon"])
        if valid_group.empty:
            continue

        m = folium.Map(
            location=[valid_group["wgs84Lat"].mean(), valid_group["wgs84Lon"].mean()],
            zoom_start=13
        )

        # 병원/약국 각각 클러스터링 및 시각화
        for category in ['병원', '약국']:
            sub = valid_group[valid_group["구분"] == category]
            if sub.empty:
                continue

            coords = sub[["wgs84Lat", "wgs84Lon"]].to_numpy()
            kms_per_radian = 6371.0088
            epsilon = 1.0 / kms_per_radian  # 1km

            # DBSCAN으로 500m 내 클러스터 생성
            db = DBSCAN(eps=epsilon, min_samples=2, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
            sub["cluster"] = db.labels_

            for cluster_id in sub["cluster"].unique():
                cluster_points = sub[sub["cluster"] == cluster_id]
                cluster_coords = cluster_points[["wgs84Lat", "wgs84Lon"]].to_numpy()

                # 중심 좌표 계산
                center_lat = cluster_coords[:, 0].mean()
                center_lon = cluster_coords[:, 1].mean()

                # 중심점으로부터 최대 거리 계산
                max_dist = max(great_circle((center_lat, center_lon), (lat, lon)).meters for lat, lon in cluster_coords)

                # 반경은 최대 거리 + 여유(10km)
                radius = max_dist + 10000

                folium.Circle(
                    location=(center_lat, center_lon),
                    radius=radius,
                    color='red' if category == '병원' else 'green',
                    weight=2,
                    fill=True,
                    fill_color='red' if category == '병원' else 'green',
                    fill_opacity=0.1
                ).add_to(m)

                # 마커 추가
                for _, row in cluster_points.iterrows():
                    folium.Marker(
                        location=(row["wgs84Lat"], row["wgs84Lon"]),
                        popup=folium.Popup(generate_popup(row), max_width=300),
                        icon=get_icon(row["구분"])
                    ).add_to(m)

        m.save(os.path.join(output_dir, f"{region}.html"))

    zip_path = output_dir + ".zip"
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in os.listdir(output_dir):
            file_path = os.path.join(output_dir, file)
            zipf.write(file_path, arcname=file)

In [11]:
os.chdir(r"C:\Users\hk522\Desktop")
print("현재 작업 디렉토리:", os.getcwd())

현재 작업 디렉토리: C:\Users\hk522\Desktop


In [9]:
group1 = full_df.groupby("Jibun")

In [13]:
def create_maps_combined(df, output_dir='캡스톤 대한민국 지도'):
    os.makedirs(output_dir, exist_ok=True)

    valid_group = df.dropna(subset=["wgs84Lat", "wgs84Lon"])
    if valid_group.empty:
        return

    # 대한민국 전체가 보이도록 초기 중심과 줌 설정
    m = folium.Map(
        location=[36.5, 127.5],
        zoom_start=7
    )

    coords = valid_group[["wgs84Lat", "wgs84Lon"]].to_numpy()
    kms_per_radian = 6371.0088
    epsilon = 0.5 / kms_per_radian  # 500m

    # 병원과 약국 통합하여 클러스터링
    db = DBSCAN(eps=epsilon, min_samples=2, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    valid_group["cluster"] = db.labels_

    for cluster_id in valid_group["cluster"].unique():
        cluster_points = valid_group[valid_group["cluster"] == cluster_id]
        cluster_coords = cluster_points[["wgs84Lat", "wgs84Lon"]].to_numpy()

        center_lat = cluster_coords[:, 0].mean()
        center_lon = cluster_coords[:, 1].mean()

        max_dist = max(great_circle((center_lat, center_lon), (lat, lon)).meters for lat, lon in cluster_coords)
        radius = max_dist + 10000  # 10km 여유

        folium.Circle(
            location=(center_lat, center_lon),
            radius=radius,
            color='blue',
            weight=2,
            fill=True,
            fill_color='blue',
            fill_opacity=0.1
        ).add_to(m)

        for _, row in cluster_points.iterrows():
            folium.Marker(
                location=(row["wgs84Lat"], row["wgs84Lon"]),
                popup=folium.Popup(generate_popup(row), max_width=300),
                icon=get_icon(row["구분"])
            ).add_to(m)

    m.save(os.path.join(output_dir, "전체_지도.html"))

In [ ]:
create_maps_for_all(full_df)

C:\Users\hk522\AppData\Local\Temp\ipykernel_16092\3240229690.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub["cluster"] = db.labels_
C:\Users\hk522\AppData\Local\Temp\ipykernel_16092\3240229690.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub["cluster"] = db.labels_


In [ ]:
create_maps_by_region(full_df, group1)

In [ ]:
create_maps_by_region(full_df, group2, output_dir='캡스톤 시군구 지도')